Joel Gritter & Kindeep Singh Kargil

COSC 4P80 - Seminar Demo

March 29, 2021

In [ ]:
# Music output
!sudo apt-get install fluidsynth
!pip install midi2audio
!pip install mingus
from mingus.containers import Note, NoteContainer, Track
from mingus.midi.midi_file_out import write_NoteContainer, write_Track
from midi2audio import FluidSynth

fsy = FluidSynth()

# imports for data manipulation
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# imports for machine learning
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libfluidsynth1 libqt5x11extras5 qsynth
Suggested packages:
  fluid-soundfont-gs timidity jackd
The following NEW packages will be installed:
  fluid-soundfont-gm fluidsynth libfluidsynth1 libqt5x11extras5 qsynth
0 upgraded, 5 newly installed, 0 to remove and 30 not upgraded.
Need to get 120 MB of archives.
After this operation, 150 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fluid-soundfont-gm all 3.1-5.1 [119 MB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libfluidsynth1 amd64 1.1.9-1 [137 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fluidsynth amd64 1.1.9-1 [20.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libqt5x11extras5 amd64 5.9.5-0ubuntu1 [8,596 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic/uni

In [ ]:
# read in the notes, make an array with 0's, except for the current note
def read_and_format(input_filepath):
  input_data = []
  with open(input_filepath) as input_file:
    for line in input_file:
      values = line.split(",")
      for value in values:
        tmp = [0.0] * 88
        v = int(value)
        tmp[v-1] = 1.0
        input_data.append(tmp)
  
  return input_data
  

input_data = read_and_format("k330-allegro-moderato.csv")

In [ ]:
# get the previous 20 notes, predict the next note
def generate_datasets(input_array, n_prev = 20):
  temp_x = [input_array[i:i+n_prev] for i in range(len(input_array) - n_prev)]
  temp_y = [input_array[i+n_prev] for i in range(len(input_array) - n_prev)]

  return np.array(temp_x), np.array(temp_y)

x, y = generate_datasets(input_data)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, shuffle=True)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
print(y_train[0])

(2008, 20, 88) (2008, 88)
(502, 20, 88) (502, 88)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
# build the model itself
model = Sequential()
model.add(LSTM(30))
model.add(Dense(88, activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# train the model
model.fit(x_train, y_train, batch_size=10, epochs=100, validation_split=0.05)

Epoch 1/100
191/191 [==============================] - 5s 13ms/step - loss: 3.6925 - accuracy: 0.0915 - val_loss: 2.9033 - val_accuracy: 0.1287
Epoch 2/100
191/191 [==============================] - 2s 10ms/step - loss: 2.6368 - accuracy: 0.1662 - val_loss: 2.8612 - val_accuracy: 0.1287
Epoch 3/100
191/191 [==============================] - 2s 10ms/step - loss: 2.6247 - accuracy: 0.1678 - val_loss: 2.8293 - val_accuracy: 0.1287
Epoch 4/100
191/191 [==============================] - 2s 10ms/step - loss: 2.5818 - accuracy: 0.1751 - val_loss: 2.7927 - val_accuracy: 0.1584
Epoch 5/100
191/191 [==============================] - 2s 10ms/step - loss: 2.5643 - accuracy: 0.2007 - val_loss: 2.7643 - val_accuracy: 0.1782
Epoch 6/100
191/191 [==============================] - 2s 10ms/step - loss: 2.5164 - accuracy: 0.2176 - val_loss: 2.7194 - val_accuracy: 0.2178
Epoch 7/100
191/191 [==============================] - 2s 10ms/step - loss: 2.4398 - accuracy: 0.2360 - val_loss: 2.6767 - val_accuracy:

In [ ]:
# test the model
model.evaluate(x_test, y_test)

16/16 [==============================] - 0s 3ms/step - loss: 1.0748 - accuracy: 0.7092


[1.0747835636138916, 0.7091633677482605]

In [ ]:
# See incorrectly predicted 
predictions = model.predict(x_test)
incorrect_indices = []
for (index, (prediction, target)) in enumerate(zip(predictions, y_test)):
  pred = np.argmax(prediction)
  tar = np.argmax(target)
  if pred != tar:
    incorrect_indices.append(index)

print(", ".join(map(str, incorrect_indices)))

1, 6, 8, 10, 11, 12, 17, 25, 29, 30, 31, 39, 42, 45, 46, 48, 58, 80, 83, 84, 86, 88, 89, 95, 97, 105, 107, 109, 111, 118, 126, 137, 140, 143, 144, 145, 146, 147, 149, 150, 152, 154, 156, 158, 160, 161, 162, 164, 167, 168, 169, 178, 180, 182, 185, 188, 192, 193, 200, 201, 202, 203, 206, 210, 212, 217, 226, 231, 236, 239, 245, 246, 247, 250, 253, 254, 257, 268, 269, 274, 277, 278, 279, 281, 285, 286, 288, 290, 292, 296, 297, 301, 303, 316, 317, 329, 330, 346, 353, 357, 359, 361, 362, 363, 365, 367, 368, 371, 377, 378, 382, 385, 387, 397, 402, 405, 407, 411, 416, 418, 421, 424, 426, 428, 429, 431, 434, 435, 443, 445, 449, 450, 452, 454, 462, 463, 465, 468, 471, 473, 478, 485, 487, 488, 498, 501


In [ ]:
# Predict song
test_in = x_test[0]
test_out = y_test[0]
# initial - provide inital 20 notes
# n - how many predicted notes to add (i.e. expand by this number)
def make_big_song(initial, n):
  res =[ x for x in initial]
  for _ in range(n):
    next = model.predict(np.array([res[-20:],]))[0]
    res.append(next)

  return np.array(res)

test = make_big_song(test_in, 60)
print(test.shape)

(80, 88)


In [ ]:
# Expects n x 88
def vector_to_midi(arr, filename="nice.midi"):
  track = Track()
  for note_arr in arr:
    note_num = int(np.argmax(note_arr))
    note = Note()
    note.from_int(note_num - 3)
    track.add_notes(note)
  write_Track(filename, track)
  print("Done!")
  
vector_to_midi(test)

Done!


In [ ]:
def predict_to_file(first_20_notes, expected, filename="nice"):
  next = model.predict(np.array([first_20_notes]))
  actual_next = np.array([expected])
  next_file = filename + "_predicted_note"
  actual_next_file = filename + "_actual_note"
  orig_file = filename + "_first_20_notes"

  vector_to_midi(next, next_file + ".midi")
  vector_to_midi(actual_next, actual_next_file + ".midi")
  vector_to_midi(first_20_notes, orig_file + ".midi")

  # This conversion not seem to work
  # fsy.midi_to_audio(next_file + ".midi", next_file + ".mp3")
  # fsy.midi_to_audio(actual_next_file + ".midi", actual_next_file + ".mp3")
  # fsy.midi_to_audio(orig_file + ".midi",  orig_file + ".mp3")

predict_to_file(test_in, test_out)

inci = incorrect_indices[0]
predict_to_file(x_test[inci], y_test[inci], 'first_incorrect')


Done!
Done!
Done!
Done!
Done!
Done!
